# Implementing logistic regression from scratch

The goal of this notebook is to implement your own logistic regression classifier. You will:

 * Extract features from Amazon product reviews.
 * Convert an pandas DataFrame into a NumPy array.
 * Implement the link function for logistic regression.
 * Write a function to compute the derivative of the log likelihood function with respect to a single coefficient.
 * Implement gradient ascent.
 * Given a set of coefficients, predict sentiments.
 * Compute classification accuracy for the logistic regression model.
 
Let's get started!

In [1]:
# Import some libs
import pandas
import numpy as np
from sklearn.model_selection import train_test_split

### Load review dataset
For this assignment, we will use a subset of the Amazon product review dataset. The subset was chosen to contain similar numbers of positive and negative reviews, as the original dataset consisted primarily of positive reviews.

Load the dataset into a data frame named **products**. One column of this dataset is **sentiment**, corresponding to the class label with +1 indicating a review with positive sentiment and -1 for negative sentiment.

Let us quickly explore more of this dataset. The **name** column indicates the name of the product. Try listing the name of the first 10 products in the dataset.

After that, try counting the number of positive and negative reviews.

In [4]:
# Import amazon_baby.csv data to pandas dataframe
products_df = pandas.read_csv('./dat/amazon_baby_subset.csv')

In [5]:
print(products_df[0:10])
print(products_df.groupby('sentiment').count())

                                                name  \
0  Stop Pacifier Sucking without tears with Thumb...   
1    Nature's Lullabies Second Year Sticker Calendar   
2    Nature's Lullabies Second Year Sticker Calendar   
3                        Lamaze Peekaboo, I Love You   
4  SoftPlay Peek-A-Boo Where's Elmo A Children's ...   
5                          Our Baby Girl Memory Book   
6  Hunnt&reg; Falling Flowers and Birds Kids Nurs...   
7  Blessed By Pope Benedict XVI Divine Mercy Full...   
8  Cloth Diaper Pins Stainless Steel Traditional ...   
9  Cloth Diaper Pins Stainless Steel Traditional ...   

                                              review  rating  sentiment  
0  All of my kids have cried non-stop when I trie...       5          1  
1  We wanted to get something to keep track of ou...       5          1  
2  My daughter had her 1st baby over a year ago. ...       5          1  
3  One of baby's first and favorite books, and it...       4          1  
4  Very cute 

### Apply text cleaning on the review data
In this section, we will perform some simple feature cleaning using data frames. The last assignment used all words in building bag-of-words features, but here we limit ourselves to 193 words (for simplicity). We compiled a list of 193 most frequent words into the JSON file named **important_words.json**. Load the words into a list **important_words**.

In [7]:
import json

with open('./dat/important_words.json', 'r') as f:
    important_words = json.loads(f.read())
print (important_words)

['baby', 'one', 'great', 'love', 'use', 'would', 'like', 'easy', 'little', 'seat', 'old', 'well', 'get', 'also', 'really', 'son', 'time', 'bought', 'product', 'good', 'daughter', 'much', 'loves', 'stroller', 'put', 'months', 'car', 'still', 'back', 'used', 'recommend', 'first', 'even', 'perfect', 'nice', 'bag', 'two', 'using', 'got', 'fit', 'around', 'diaper', 'enough', 'month', 'price', 'go', 'could', 'soft', 'since', 'buy', 'room', 'works', 'made', 'child', 'keep', 'size', 'small', 'need', 'year', 'big', 'make', 'take', 'easily', 'think', 'crib', 'clean', 'way', 'quality', 'thing', 'better', 'without', 'set', 'new', 'every', 'cute', 'best', 'bottles', 'work', 'purchased', 'right', 'lot', 'side', 'happy', 'comfortable', 'toy', 'able', 'kids', 'bit', 'night', 'long', 'fits', 'see', 'us', 'another', 'play', 'day', 'money', 'monitor', 'tried', 'thought', 'never', 'item', 'hard', 'plastic', 'however', 'disappointed', 'reviews', 'something', 'going', 'pump', 'bottle', 'cup', 'waste', 'retu

Here we remove **remove_punctuation** and fill in N/A's with empty reivew

In [8]:
def remove_punctuation(text):
    import string
    return text.translate(text.maketrans('', '', string.punctuation))

products_df = products_df.fillna({'review':''})  # fill in N/A's in the review column
products_df['review_clean'] = products_df['review'].apply(remove_punctuation)

Now we proceed with the second item. For each word in **important_words**, we compute a count for the number of times the word occurs in the review. We will store this count in a separate column (one for each word). The result of this feature processing is a single column for each word in **important_words** which keeps a count of the number of times the respective word occurs in the review text.

In [9]:
from sklearn.feature_extraction.text import CountVectorizer
import  pandas as pd
# Initialize vectorizer to only look for your important words
vectorizer = CountVectorizer(vocabulary=important_words)

# Transform the text into a sparse matrix
count_matrix = vectorizer.fit_transform(products_df['review_clean'])

# Convert back to a DataFrame and join
count_df = pd.DataFrame(count_matrix.toarray(), columns=important_words)
products_df = pd.concat([products_df, count_df], axis=1)

In [10]:
products_df.head()

,name,review,rating,sentiment,review_clean,baby,one,great,love,use,...,seems,picture,completely,wish,buying,babies,won,tub,almost,either
0,Stop Pacifier Sucking without tears with Thumb...,All of my kids have cried non-stop when I trie...,5,1,All of my kids have cried nonstop when I tried...,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Nature's Lullabies Second Year Sticker Calendar,We wanted to get something to keep track of ou...,5,1,We wanted to get something to keep track of ou...,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Nature's Lullabies Second Year Sticker Calendar,My daughter had her 1st baby over a year ago. ...,5,1,My daughter had her 1st baby over a year ago S...,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,"Lamaze Peekaboo, I Love You","One of baby's first and favorite books, and it...",4,1,One of babys first and favorite books and it i...,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,SoftPlay Peek-A-Boo Where's Elmo A Children's ...,Very cute interactive book! My son loves this ...,5,1,Very cute interactive book My son loves this b...,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


Now, write some code to compute the number of product reviews that contain the word perfect.
<br>
**Quiz Question:** How many reviews contain the word **"perfect"**?
<br>
**Your answer:** 

In [11]:
# YOUR CODE HERE
# print(products_df[products_df.perfect > 0].perfect.count())
sum(1 for x in products_df['perfect'] if x > 0)
# YOUR CODE HERE

3309

### Convert data frame to multi-dimensional array
Write a function that extracts columns from a data frame and converts them into a multi-dimensional array. We plan to use them throughout the course, so make sure to get this function right.
<br>
The function should accept three parameters:
<br>
- **dataframe:** a data frame to be converted
- **features:** a list of string, containing the names of the columns that are used as features.
- **label:** a string, containing the name of the single column that is used as class labels.
<br>
<br>
The function should return two values:
<br>
- one 2D array for features
- one 1D array for class labels
<br>
<br>
The function should do the following:
<br>
- Prepend a new column constant to dataframe and fill it with 1's. This column takes account of the intercept term. Make sure that the constant column appears first in the data frame.
- Prepend a string 'constant' to the list features. Make sure the string 'constant' appears first in the list.
- Extract columns in dataframe whose names appear in the list features.
- Convert the extracted columns into a 2D array using a function in the data frame library. If you are using Pandas, you would use as_matrix() function.
- Extract the single column in dataframe whose name corresponds to the string label.
- Convert the column into a 1D array.
- Return the 2D array and the 1D array.

In [12]:
def get_numpy_data(dataframe, features, label):
    dataframe['constant'] = 1
    features = ['constant'] + features
    features_frame = dataframe[features]
    feature_matrix = features_frame.values
    label_array = dataframe[label].values
    return (feature_matrix, label_array)

Using the function **get_numpy_data**, extract two arrays **feature_matrix** and **sentiment**. The 2D array **feature_matrix** would contain the content of the columns given by the list **important_words**. The 1D array sentiment would contain the content of the column **sentiment**.

In [13]:
feature_matrix, sentiment = get_numpy_data(products_df, important_words, 'sentiment')

**Quiz Question:** How many features are there in the feature_matrix?
<br>
**Your answer:**

In [14]:
# YOUR CODE HERE
np.shape(feature_matrix)[1]
# YOUR CODE HERE

194

Recall from lecture that the link function is given by
<br>
$P(y_i = +1 | \mathbf{x}_i, \mathbf{w}) = \dfrac{1}{1 + \exp{(-\mathbf{w}^\intercal h(\mathbf{x}_i))}}$
<br>
where the feature vector $h(\mathbf{x}_i)$ represents the word counts of **important_words** in the review $\mathbf{x}_i$
<br>
Write a function named **predict_probability** that implements the link function.
<br>
- Take two parameters: **feature_matrix** and **coefficients**.
- First compute the dot product of **feature_matrix** and **coefficients**.
- Then compute the link function $P(y = +1 | \mathbf{x}, \mathbf{w})$
- Return the predictions given by the link function.

In [15]:
'''
produces probablistic estimate for P(y_i = +1 | x_i, w).
estimate ranges between 0 and 1.
'''
def predict_probability(feature_matrix, coefficients):
    # Take dot product of feature_matrix and coefficients  
    # YOUR CODE HERE
    scores = np.dot(feature_matrix, coefficients)
    # Compute P(y_i = +1 | x_i, w) using the link function
    predictions = 1/(1+np.exp(-scores))
    # return predictions
    return scores, predictions

**Aside**. How the link function works with matrix algebra

Since the word counts are stored as columns in **feature_matrix**, each i-th row of the matrix corresponds to the feature vector $h(\mathbf{x}_i)$:
$$[\mbox{feature_matrix}] = \left[\begin{array}{c} h(\mathbf{x}_1)^\intercal \\ h(\mathbf{x}_2)^\intercal \\ \vdots \\ h(\mathbf{x}_N)^\intercal\end{array}\right] = \left[\begin{array}{cccc}h_0(\mathbf{x}_1) & h_1(\mathbf{x}_1) & \cdots & h_D(\mathbf{x}_1) \\ h_0(\mathbf{x}_2) & h_1(\mathbf{x}_2) & \cdots & h_D(\mathbf{x}_2) \\ \vdots & \vdots & \ddots & \vdots \\ h_0(\mathbf{x}_N) & h_1(\mathbf{x}_N) & \cdots & h_D(\mathbf{x}_N) \end{array}\right]$$
By the rules of matrix multiplication, the score vector containing elements $\mathbf{w}^\intercal h(\mathbf{x}_i)$ is obtained by multiplying **feature_matrix** and the coefficient vector $\mathbf{w}$:
$$[\mbox{score}] = [\mbox{feature_matrix}]\mathbf{w} = \left[\begin{array}{c} h(\mathbf{x}_1)^\intercal \\ h(\mathbf{x}_2)^\intercal \\ \vdots \\ h(\mathbf{x}_N)^\intercal\end{array}\right] \mathbf{w} = \left[\begin{array}{c} h(\mathbf{x}_1)^\intercal \mathbf{w} \\ h(\mathbf{x}_2)^\intercal \mathbf{w} \\ \vdots \\ h(\mathbf{x}_N)^\intercal \mathbf{w}\end{array}\right] = \left[\begin{array}{c} \mathbf{w}^\intercal h(\mathbf{x}_1) \\ \mathbf{w}^\intercal h(\mathbf{x}_2) \\ \vdots \\ \mathbf{w}^\intercal h(\mathbf{x}_N) \end{array}\right]$$

### Compute derivative of log likelihood with respect to a single coefficient
Recall from lecture:
$$\displaystyle \frac{\partial \ell}{\partial w_j} = \sum_{i=1}^N h_j(\mathbf{x}_i) (\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w}))$$
- **errors:** vector whose i-th value contains
$$\mathbf{1}[y_i = +1] - P(y_i = +1 | \mathbf{x}_i, \mathbf{w})$$
- **feature:** vector whose i-th value contains
$h_j(\mathbf{x}_i)$
<br>
This corresponds to the j-th column of feature_matrix.
<br>
The function should do the following:
<br>
- Take two parameters **errors** and **feature**.
- Compute the dot product of **errors** and **feature**.
- Return the dot product. This is the derivative with respect to a single coefficient w_j.

In [16]:
def feature_derivative(errors, feature):     
    # YOUR CODE HERE
    # Compute the dot product of errors and feature
    derivative = np.dot(errors, feature)
    # Return the derivative
    # YOUR CODE HERE
    return derivative

In the main lecture, our focus was on the likelihood. In the advanced optional video, however, we introduced a transformation of this likelihood---called the log-likelihood---that simplifies the derivation of the gradient and is more numerically stable. Due to its numerical stability, we will use the log-likelihood instead of the likelihood to assess the algorithm.

The log-likelihood is computed using the following formula (see the advanced optional video if you are curious about the derivation of this equation):
$$\displaystyle \ell \ell (\mathbf{w}) = \sum_{i=1}^N \Big( (\mathbf{1}[y_i = +1] - 1) \mathbf{w}^\intercal h(\mathbf{x}_i) - \ln{\big(1 + \exp{(-\mathbf{w}^\intercal h(\mathbf{x}_i) )} \big)} \Big)$$
<br>
Write a function **compute_log_likelihood** that implements the equation
<br>
The function has two parameters:
<br>
**indicator**: Has shape (N, 1). **indicator[i]** = True if **yi** = +1 otherwise **indicator[i]** = False
<br>
**scores**: The scores return by the **predict_probability** function. Refer to the above section for more details about the **scores** parameter.

In [17]:
def compute_log_likelihood(indicator, scores):
    lp = np.sum((indicator-1)*scores - np.log(1. + np.exp(-scores)))
    return lp

### Taking gradient steps
Now we are ready to implement our own logistic regression. All we have to do is to write a gradient ascent function that takes gradient steps towards the optimum.
<br>
Write a function **logistic_regression** to fit a logistic regression model using gradient ascent.
<br>
The function accepts the following parameters:
<br>
- **feature_matrix:** 2D array of features
- **sentiment:** 1D array of class labels
- **initial_coefficients:** 1D array containing initial values of coefficients
- **step_size:** a parameter controlling the size of the gradient steps
- **max_iter:** number of iterations to run gradient ascent
<br>
The function returns the last set of coefficients after performing gradient ascent.
<br>
The function carries out the following steps:
<br>
1. Initialize vector **coefficients** to **initial_coefficients**.
2. Predict the class probability $P(y_i = +1 | \mathbf{x}_i,\mathbf{w})$ using your **predict_probability** function and save it to variable **predictions**.
3. Compute indicator value for $(y_i = +1)$ by comparing **sentiment** against +1. Save it to variable **indicator**.
4. Compute the errors as difference between **indicator** and **predictions**. Save the errors to variable **errors**.
5. For each j-th coefficient, compute the per-coefficient derivative by calling **feature_derivative** with the j-th column of **feature_matrix**. Then increment the j-th coefficient by (step_size*derivative).
6. Once in a while, insert code to print out the log likelihood.
7. Repeat steps 2-6 for **max_iter** times.

In [18]:
from math import sqrt
def logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter):
    coefficients = initial_coefficients.copy() # make sure it's a numpy array
    for itr in range(max_iter):
        # Predict P(y_i = +1|x_1,w) using your predict_probability() function
        # YOUR CODE HERE
        scores, predictions = predict_probability(feature_matrix, coefficients)
        # YOUR CODE HERE

        # Compute indicator value for (y_i = +1)
        indicator = (sentiment==+1)

        # Compute the errors as indicator - predictions
        errors = indicator - predictions

        for j in range(coefficients.shape[0]): # loop over each coefficient
            # Recall that feature_matrix[:,j] is the feature column associated with coefficients[j]
            # compute the derivative for coefficients[j]. Save it in a variable called derivative
            # YOUR CODE HERE
            derivative = feature_derivative(errors, feature_matrix[:,j])
            # YOUR CODE HERE

            # add the step size times the derivative to the current coefficient
            # YOUR CODE HERE
            coefficients[j] = coefficients[j] + step_size*derivative
            # YOUR CODE HERE
        
        if itr % 100 == 0:
            lp = compute_log_likelihood(indicator, scores)
            print ('iteration: {}, log likelihood: {}'.format(itr, lp))

    return coefficients

Now, let us run the logistic regression solver with the parameters below:
<br>
- **feature_matrix** = feature_matrix extracted
- **sentiment** = sentiment extracted
- **initial_coefficients** = a 194-dimensional vector filled with zeros
- **step_size** = 1e-7
- **max_ite**r = 301
<br>
Save the returned **coefficients** to variable **coefficients**.

In [19]:
initial_coefficients = np.zeros((feature_matrix.shape[1]))
step_size = 1e-7
max_iter = 301

In [20]:
coefficients = logistic_regression(feature_matrix, sentiment, initial_coefficients, step_size, max_iter)

iteration: 0, log likelihood: -36786.70716667741
iteration: 100, log likelihood: -36122.13258712315
iteration: 200, log likelihood: -35520.61731276558
iteration: 300, log likelihood: -34972.62003352713


**Quiz question:** As each iteration of gradient ascent passes, does the log likelihood increase or decrease?
<br>
**Your answer:** increase

### Predicting sentiments
Recall from lecture that class predictions for a data point x can be computed from the coefficients w using the following formula:
<br>
$$\hat{y}_i = \begin{cases} +1 & \text{if }\mathbf{x}_i^\intercal \mathbf{w} > 0 \\ -1 & \text{if }\mathbf{x}_i^\intercal \mathbf{w} \leq 0\end{cases}$$
Now, we write some code to compute class predictions. We do this in two steps:
- First compute the scores using feature_matrix and coefficients using a dot product.
- Then apply threshold 0 on the scores to compute the class predictions. Refer to the formula above.
<br>

**Quiz question:** How many reviews were predicted to have positive sentiment?
<br>
**Your answer:**

In [21]:
scores = feature_matrix.dot(coefficients)

In [22]:
# YOUR CODE HERE
pred_sentiments = [1 if x > 0 else -1 for x in scores]
print("Number of positive sentiments: {}".format(sum(1 for x in pred_sentiments if x == 1)))
# YOUR CODE HERE

Number of positive sentiments: 25237


### Measuring accuracy
We will now measure the classification accuracy of the model. Recall from the lecture that the classification accuracy can be computed as follows:
$$\mbox{accuracy} = \dfrac{\mbox{# correctly classified data points}}{\mbox{# total data points}}$$
**Quiz question:** hat is the accuracy of the model on predictions made above? (round to 2 digits of accuracy)
<br>
**Your answer:**

In [23]:
len(diff)

NameError: name 'diff' is not defined

In [ ]:
diff = (scores > 0) == (sentiment > 0)
print ('Accuracy: {}'.format(np.sum(diff) / len(diff)))

Recall that in the earlier assignment, we were able to compute the "**most positive words**". These are words that correspond most strongly with positive reviews. In order to do this, we will first do the following:
- Treat each coefficient as a tuple, i.e. (**word, coefficient_value**). The intercept has no corresponding word, so throw it out.
- Sort all the (**word, coefficient_value**) tuples by **coefficient_value** in descending order. Save the sorted list of tuples to **word_coefficient_tuples**.

Now, **word_coefficient_tuples** contains a sorted list of (**word, coefficient_value**) tuples. The first 10 elements in this list correspond to the words that are most positive.
<br>
**Quiz question:** What is the top 10 positive words?
<br>
**Your answer:**
<br>
**Quiz question:** What is the top 10 negative words?
<br>
**Your answer:**

In [ ]:
coefficients = list(coefficients[1:]) # exclude intercept
word_coefficient_tuples = [(word, coefficient) for word, coefficient in zip(important_words, coefficients)]
word_coefficient_tuples = sorted(word_coefficient_tuples, key=lambda x:x[1], reverse=True)


In [ ]:
# YOUR CODE
print("Top 10 positive words: {}".format(np.array(word_coefficient_tuples)[:10, 0]))
print("Top 10 negative words: {}".format(np.array(word_coefficient_tuples)[-10:, 0]))
# YOUR CODE